In [58]:
import pickle
import numpy as np

In [60]:
outFile = 'src/dglke/ckpts/TransR_CCS_2/CCS_TransR_entity.npy'

In [61]:
ent_embed = np.load(outFile)

In [63]:
ent_embed.shape

(15800, 200)

In [5]:
relation = {
    'is_parent_of': 0,
    'is_child_of': 1
}
train_set = set()
cat1count = 0
cat2count = 0
cat3count = 0
cat4count = 0

infd = open(graph_file, 'r')
_ = infd.readline()

# build dictionary of entities
for line in infd:
    tokens = line.strip().split(',')
    icd9 = tokens[0][1:-1].strip()
    cat1 = tokens[1][1:-1].strip()
    desc1 = 'A_L1_' + cat1
    cat2 = tokens[3][1:-1].strip()
    desc2 = 'A_L2_' + cat2
    cat3 = tokens[5][1:-1].strip()
    desc3 = 'A_L3_' + cat3
    cat4 = tokens[7][1:-1].strip()
    desc4 = 'A_L4_' + cat4

    if icd9.startswith('E'):
        if len(icd9) > 4: icd9 = icd9[:4] + '.' + icd9[4:]
    else:
        if len(icd9) > 3: icd9 = icd9[:3] + '.' + icd9[3:]
    icd9 = 'D_' + icd9

    if icd9 not in entities:
        entities[icd9] = len(entities)

    if desc1 not in entities:
        cat1count += 1
        entities[desc1] = len(entities)

    if len(cat2) > 0:
        if desc2 not in entities:
            cat2count += 1
            entities[desc2] = len(entities)
    if len(cat3) > 0:
        if desc3 not in entities:
            cat3count += 1
            entities[desc3] = len(entities)
    if len(cat4) > 0:
        if desc4 not in entities:
            cat4count += 1
            entities[desc4] = len(entities)
infd.close()

# add root_code
entities['A_ROOT'] = len(entities)
rootCode = entities['A_ROOT']

In [6]:
infd = open(graph_file, 'r')
infd.readline()

for line in infd:
    tokens = line.strip().split(',')
    icd9 = tokens[0][1:-1].strip()
    cat1 = tokens[1][1:-1].strip()
    desc1 = 'A_L1_' + cat1
    cat2 = tokens[3][1:-1].strip()
    desc2 = 'A_L2_' + cat2
    cat3 = tokens[5][1:-1].strip()
    desc3 = 'A_L3_' + cat3
    cat4 = tokens[7][1:-1].strip()
    desc4 = 'A_L4_' + cat4

    if icd9.startswith('E'):
        if len(icd9) > 4: icd9 = icd9[:4] + '.' + icd9[4:]
    else:
        if len(icd9) > 3: icd9 = icd9[:3] + '.' + icd9[3:]
    icd9 = 'D_' + icd9

    icdCode = entities[icd9]

    if len(cat4) > 0:
        code4 = entities[desc4]
        code3 = entities[desc3]
        code2 = entities[desc2]
        code1 = entities[desc1]
        train_set.add(str(rootCode) + ' 0 ' + str(code1))
        train_set.add(str(code1) + ' 1 ' + str(rootCode))
        train_set.add(str(code1) + ' 0 ' + str(code2))
        train_set.add(str(code2) + ' 1 ' + str(code1))
        train_set.add(str(code2) + ' 0 ' + str(code3))
        train_set.add(str(code3) + ' 1 ' + str(code2))
        train_set.add(str(code3) + ' 0 ' + str(code4))
        train_set.add(str(code4) + ' 1 ' + str(code3))
        train_set.add(str(code4) + ' 0 ' + str(icdCode))
        train_set.add(str(icdCode) + ' 1 ' + str(code4))

    elif len(cat3) > 0:
        code3 = entities[desc3]
        code2 = entities[desc2]
        code1 = entities[desc1]

        train_set.add(str(rootCode) + ' 0 ' + str(code1))
        train_set.add(str(code1) + ' 1 ' + str(rootCode))
        train_set.add(str(code1) + ' 0 ' + str(code2))
        train_set.add(str(code2) + ' 1 ' + str(code1))
        train_set.add(str(code2) + ' 0 ' + str(code3))
        train_set.add(str(code3) + ' 1 ' + str(code2))
        train_set.add(str(code3) + ' 0 ' + str(icdCode))
        train_set.add(str(icdCode) + ' 1 ' + str(code3))

    elif len(cat2) > 0:
        code2 = entities[desc2]
        code1 = entities[desc1]
        train_set.add(str(rootCode) + ' 0 ' + str(code1))
        train_set.add(str(code1) + ' 1 ' + str(rootCode))
        train_set.add(str(code1) + ' 0 ' + str(code2))
        train_set.add(str(code2) + ' 1 ' + str(code1))
        train_set.add(str(code2) + ' 0 ' + str(icdCode))
        train_set.add(str(icdCode) + ' 1 ' + str(code2))

    else:
        code1 = entities[desc1]

        train_set.add(str(rootCode) + ' 0 ' + str(code1))
        train_set.add(str(code1) + ' 1 ' + str(rootCode))
        train_set.add(str(code1) + ' 0 ' + str(icdCode))
        train_set.add(str(icdCode) + ' 1 ' + str(code1))

In [65]:
len(entities)

15800

In [67]:
leaves = 0
for key in entities.keys():
    if not key.startswith('A_'):
        leaves += 1

In [68]:
leaves

15072

In [15]:
outfd = open(outFile+'relation.dict', 'w')
for k,v in relation.items():
    outfd.write(k + ' ' + str(v) + '\n')
outfd.close()

In [12]:
outfd = open(outFile+'entities.dict', 'w')
for k,v in entities.items():
    outfd.write(k + ' ' + str(v) + '\n')
outfd.close()

In [14]:
outfd = open(outFile+'all_edges.tsv', 'w')
for edge in train_set:
    outfd.write(edge + '\n')
outfd.close()

In [56]:
import os
path = '/home/xpeng/research/projects/medicalAI_torch/src/KG_medical/outputs/KG/'
f = 'entity2id'
assert os.path.exists(os.path.join(path, f)), \
                'File {} now exist in {}'.format(f, path)